# Scrapper de sites

> Récupère le contenu de chacun des URL classé par typologie (variable ndf dans le code), chargé depuis le fichier Sites.json issu du dossier ressources.

> Stocke le résultat dans ndf qui conditionne la variable "fichier de sortie" : un fichier par catégorie de site. Celle-ci peut être adaptée pour pointer sur une zone correcte de votre drive dans la zone "Personnalisation" dénotée ci-dessous. Les données des sites collectés sont stockées au format pickle dans le dossier /OUT/ContenusSites classé par typologie selon Sites.json.

> La variable de sortie est de type dictionnaire pour conserver les url sources (flexibilité pour pouvoir corriger). Cf. commentaires infra. 




In [ ]:
# Il faut initialiser le dossier à partir de GitHub
# cette action est à faire une seule fois lors de la première utilisation
!git clone https://github.com/Patent2net/LexiComWeb.git

Cloning into 'LexiComWeb'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 38 (delta 6), reused 37 (delta 5), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [ ]:
stockageEntree = "/content/LexiComWeb/"
stockageSortie = "/content/LexiComWeb/"

In [ ]:
import os
os.chdir("LexiComWeb/RESSOURCES")
from outils import isPartner, text_from_html, myRequest


---
# Personnalisation (Option)

> Ces cellules permettent de réaliser les traitements à partir de son propre espace de stockage. A n'exécuter que dans ce cas en adaptant les dossiers d'entrée et sortie. NE PAS EXECUTER SAUF A PERSONNALISER LES TRAITEMENTS

Si vous voulez exécuter ce notebook sur vos propres données il faut reconstituer l'ensemble des dossiers nécessaires sur votre drive dans un dossier OUT situé à la racine (cf. les variables *stockageEntree* et *stockageSortie*), ces cellules vous permettent de configurer cet outil pour travailler sur vos données et non celles issues du git (accessibles sur un dossier virtuel via le menu *Fichiers* à gauche)..

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Récupérer les sorties sur son drive (créer un dossier "OUT")
stockageSortie = "/content/drive/MyDrive/OUT"

In [ ]:
# Récupérer les entrées sur son drive
stockageEntree = "/content/drive/MyDrive/OUT"

---

# Chargement des librairies

In [ ]:
import requests, re, pickle, urllib
from bs4 import BeautifulSoup

# Chargement des URL à traiter 
> les urls à collecter sont classés dans une typologie de sites 

In [ ]:
import json
with open(stockageEntree + '/RESSOURCES/Sites.json', 'r', encoding='utf-8') as f:
    CatSites = json.load(f)

In [ ]:
nb = 0
for truc in CatSites .keys():
 nb += len( CatSites [truc][0])
 print (truc, nb, CatSites [truc] [0])

gouv 6 ['http://ofb.gouv.fr', 'http://agriculture.gouv.fr', 'http://ecologique-solidaire.gouv.fr', 'http://culture.gouv.fr', 'http://developpement-durable.gouv.fr', 'http://diplomatie.gouv.fr']
parcs 16 ['http://www.vanoise-parcnational.fr/fr', 'http://www.pyrenees-parcnational.fr/fr', 'http://www.cevennes-parcnational.fr/fr', 'http://www.ecrins-parcnational.fr/', 'http://www.mercantour-parcnational.fr/fr', 'http://www.guadeloupe-parcnational.fr/fr', 'http://www.parc-amazonien-guyane.fr/fr', 'http://www.reunion-parcnational.fr/fr', 'http://www.calanques-parcnational.fr/fr', 'http://www.forets-parcnational.fr/fr']
orga 37 ['http://ademe.fr', 'http://ffcam.fr', 'http://parcsetjardins.fr', 'http://ffrandonnee.fr', 'http://fondation-patrimoine.org', 'http://iucn.org', 'http://onf.fr', 'http://ffem.fr', 'http://sanctuaire-pelagos.org', 'http://fondationbiodiversite.fr', 'http://eaurmc.fr', 'http://sfepm.org', 'http://ffe.com', 'http://atout-france.fr', 'http://naturefrance.fr', 'http://fond

In [ ]:
print (nb)

379


# Première boucle pour lever les problèmes

In [ ]:
import urllib3
urllib3.disable_warnings() # pourquoi les certificats SSL ne passent pas aujourd'hui ? 30/11 

> La dynamique du web fait que certains sites peuvent ne pas être accessibles, ne pas répondre à un instant t, ou faire planter le collecteur. Ce qui suit teste chaque URL et construit la variable BadUrl avec les urls en erreur.

changements 01/22: la fonction myRequest implémente :
1. la gestion du timeout (3 secondes), le collecteur ne devrait plus planter
2. de réssayer 3 fois en cas d'erreurs particulières (429, 502, 501)
3. ✋ ne cherche pas à vérifier le certificat SSL suite aux erreurs rencontrées ces jours ci ❗ 

In [ ]:
BadUrl = dict()
Done = []
for ndc in CatSites.keys():
  if ndc not in BadUrl.keys() and ndc not in Done:
    BadUrl[ndc] = []
    print(ndc)
    for url in CatSites[ndc][0]: 
      try:
        webpage = myRequest(url)
      except:
        print("bad", url)
        BadUrl[ndc].append(url)

gouv
bad http://ofb.gouv.fr
parcs
orga
bad http://eaurmc.fr
oTourism
bad http://visitvar.fr
Services
bad http://spinout.fr
Hebergements
bad http://campingartaudois.com
bad http://parc.fr
restauration
bad http://lemediterranee-hyeres.fr
locations
mairies


In [ ]:
# Sauvegarde
fichierDeSortie = stockageSortie + '/ContenusSites/' + 'BadUrls.json'
with open (fichierDeSortie, 'w', encoding='utf-8') as  fictemp: #on met tous les contenus dans un jspn
  json .dump(BadUrl, fictemp)


# Collecte des données de sites

In [ ]:
done = []  
# pourrait être initialisé par un os.listdir() sur le répertoire de sortie
     
for ndc in CatSites.keys():
  if ndc not in BadUrl.keys():
    BadUrl[ndc] = []
  if ndc not in done:
    soup = dict() # changement de type de données pour conserver l'URL source
  # récupère l'URL d'un site web et enregistre la page web
    for url in CatSites[ndc][0]: 
      if url not in BadUrl[ndc]:
        try:
          webpage = myRequest(url)
        except:
          BadUrl[ndc].append(url)
          continue
      else:
        continue
  # récupère le contenu de la page web à l'aide de BeautifulSoup
    #soup.append(BeautifulSoup(webpage.content, "html.parser"))
      tempoSoup = BeautifulSoup(webpage.content, "html.parser")
      if tempoSoup.title is not None:
        titre = tempoSoup.title.text 
      else:
        titre = ""
      if tempoSoup.description is not None:
        desc = tempoSoup.description.text 
      else:
        desc = ""
      texte = text_from_html(webpage.content)
      if len(titre)>0 and len(desc)>0:
        soup[url] = titre + '\n' + desc + "\n" + texte
      elif len(titre)>0:
        soup[url] = titre + "\n" + texte
      elif len(desc)>0:
        soup[url] = desc + "\n" + texte
      else:
        soup[url] = texte

      # le contenu est dans le "casier" nommé par l'url
    # nettoyage
    for cle in soup.keys():
    # soup [cle] = re.sub('^a-zA-Z0-9àâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ', ' ', soup [cle])
      re.sub('^a-zA-ZàâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ', ' ', soup [cle])
      soup[cle] = soup[cle].replace("\xa0", " ")
      soup[cle] = soup[cle].replace("\n", " ")
      soup[cle] = soup[cle].replace("’", "'")
      soup[cle] = soup[cle].translate('utf8')
      dictionary = {"\\": ""}
      transtable= soup[cle].maketrans(dictionary)
      soup[cle] = soup[cle].translate(transtable)
      soup[cle] = str(soup[cle])
    
      # sauvegarde sur Google Drive
    #fichierDeSortie = stockageSortie + '/ContenusSites/' +CatSites [ndc][1] +'.pkl'
    #with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans un pickle
    #  pickle.dump(soup, fictemp)
    fichierDeSortie = stockageSortie + '/ContenusSites/' +CatSites [ndc][1] +'.json'
    with open (fichierDeSortie, 'w', encoding='utf-8') as  fictemp: #on met tous les contenus dans un pickle
      json .dump(soup, fictemp)
    done .append (ndc)

# Commentaires et exemples de ce qui est récupéré


## BadUrl contient tous les url en erreur
> Ceux-ci sont rangés par catégorie d'origine. 

In [ ]:
import pprint as pp 

pp.pprint (BadUrl)

## Le collecteur produit un dictionnaire 
*   les clés du dictionnaire de sortie sont les url
*   Le contenu du dictionnaire pour une clé donne le texte récupéré

In [ ]:
soup.keys()

In [ ]:
soup ['http://la-seyne.fr']